In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from fastai.vision import *
from fastai import *
from pathlib import Path
import json
import collections
import numpy as np

In [ ]:
path = Path('/opt/data/pascal2007/')
img_path = path/'VOCdevkit/VOC2007/JPEGImages'

trn_path = path/'PASCAL_VOC/pascal_train2007.json'
trn_images, trn_lbl_bbox = get_annotations(trn_path)
trn_img2bbox = dict(zip(trn_images, trn_lbl_bbox))

val_path = path/'PASCAL_VOC/pascal_val2007.json'
val_images, val_lbl_bbox = get_annotations(val_path)
val_img2bbox = dict(zip(val_images, val_lbl_bbox))

img2bbox = {**trn_img2bbox, **val_img2bbox}

def get_y_func(o):
    x = img2bbox[o.name]
    return x

In [ ]:
# Use 1/10 of dataset
# I tried using use_partial_data(0.1) but this made .label_from_func crash.
data = ObjectItemList.from_folder(img_path)
num_items = round(len(data) / 10)
data = data[:num_items]

In [ ]:
data = data.split_by_files(val_images) \
        .label_from_func(get_y_func) \
        .transform(get_transforms(), tfm_y=True, size=224) \
        .databunch(bs=16, collate_fn=bb_pad_collate) \
        .normalize(imagenet_stats)

In [ ]:
data.show_batch()